# Sum Frame Sudoku puzzle

Fill in numbers from 1 to 9 so that each row, column and 3x3 block contains each number exactly once.
Numbers in the outside frame equal the sum of the first three numbers in the corresponding row or column
in the given direction.

Find more examples [here](http://frame-sudoku.blogspot.com)

Imports

In [1]:
from ortools.constraint_solver import pywrapcp
from itertools import product
import numpy as np

Pretty-print board

In [2]:
def pretty_print(board):
    for i in range(len(board)):
        for j in range(len(board)):
            print("[{}] ".format(board[i][j].Value()), end='')
        print("\n")
    print("\n\n")

Sudoku puzzle from slides

In [3]:
#top = [15, 18, 12, 11, 21, 13, 15, 17, 13]
#right = [22, 8, 15, 22, 12, 11, 15, 13, 17]
#bottom = [15, 9, 21, 10, 16, 19, 13, 15, 17]
#left = [8, 15, 22, 11, 13, 21, 18, 19, 8]

And another example (commented out)

In [4]:
top = [21, 12, 12, 13, 14, 18, 10, 19, 16]
right = [20, 15, 10, 22, 8, 15, 17, 15, 13]
bottom = [17, 9, 19, 18, 13, 14, 23, 15, 7]
left = [12, 12, 21, 14, 14, 17, 14, 9, 22]

Create constraint solver

In [5]:
solver = pywrapcp.Solver("Sum Frame Sudoku")

In [6]:
# Create Grid
grid = [[solver.IntVar(1, 9) for _j in range(9)] for _i in range(9)]

print("rules for rows, columns...")
for i in range(9):
    solver.Add(solver.AllDifferent([grid[i][j] for j in range(9)])) # Rows 
    solver.Add(solver.AllDifferent([grid[j][i] for j in range(9)])) # Columns


print("rules for 3x3 blocks...")
cell_indices = list(product(range(3), repeat=2))
    
for i, j in cell_indices: 
    solver.Add(solver.AllDifferent(
        [grid[i * 3 + di][j * 3 + dj]
                  for di in range(3)
                  for dj in range(3)]))


print("rules for top, right, bottom & left sums...")
#rules left
for i in range(9):
    sum = 0
    for j in range(3):
        sum = sum + grid[i][j]
    solver.Add(left[i] == sum)
    
#rules right
for i in range(9):
    sum = 0
    for j in range(3):
        sum = sum + grid[i][6+j]
    solver.Add(right[i] == sum)
    
#rules top
for i in range(9):
    sum = 0
    for j in range(3):
        sum = sum + grid[j][i]
    solver.Add(top[i] == sum)
    
#rules bottom
for i in range(9):
    sum = 0
    for j in range(3):
        sum = sum + grid[8-j][i]
    solver.Add(bottom[i] == sum)

rules for rows, columns...
rules for 3x3 blocks...
rules for top, right, bottom & left sums...


Configure solver

In [7]:
# Replace this by a list of decision variables
all_vars = list(np.concatenate(grid))

db = solver.Phase(all_vars, solver.INT_VAR_SIMPLE, solver.INT_VALUE_SIMPLE)

Start solver

In [8]:
solver.NewSearch(db)
while solver.NextSolution():
    pretty_print(grid)

[6] [5] [1] [2] [8] [3] [4] [9] [7] 

[7] [3] [2] [4] [5] [9] [1] [8] [6] 

[8] [4] [9] [7] [1] [6] [5] [2] [3] 

[1] [8] [5] [3] [2] [4] [7] [6] [9] 

[4] [7] [3] [6] [9] [8] [2] [1] [5] 

[2] [9] [6] [5] [7] [1] [3] [4] [8] 

[5] [1] [8] [9] [3] [2] [6] [7] [4] 

[3] [2] [4] [8] [6] [7] [9] [5] [1] 

[9] [6] [7] [1] [4] [5] [8] [3] [2] 






Cleanup

In [9]:
solver.EndSearch()

Print solver information

In [10]:
print("Solutions: {}".format(solver.Solutions()))
print("Runtime:   {}ms".format(solver.WallTime()))
print("Failures:  {}".format(solver.Failures()))
print("Branches:  {} ".format(solver.Branches()))

Solutions: 1
Runtime:   173ms
Failures:  463
Branches:  926 
